# Study Case:  (X)

In [1]:
import mysql.connector
import operator

import numpy as np
import pandas as pd

import datetime

import matplotlib as mlp
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

#import plotly as py
import plotly.offline as po
import plotly.graph_objs as go
import plotly.express as px

import ipywidgets as widgets
# import seaborn as sns


In [2]:
%matplotlib inline 
po.init_notebook_mode(connected=True)

In [3]:
 pd.set_option("display.max_rows", 5000)                      # To display more rows

In [4]:
 pd.reset_option("^display")                                  # Reset the number of rows displayed to default

- **Number of metabolites**

We'll use a query so that we can retrieve the metabolites that exist on **bioanalysis**. This is important for eliminating the metabolite duplicates on the following dataframe.

In [5]:
# Retrieve all bioanalysis_metabolite table from DB

cnx = mysql.connector.connect(user='root', password='root',
                              host='127.0.0.1',
                              database='bioeco')

sql_query = pd.read_sql_query(''' select * from bioanalysis_metabolite ''', cnx, parse_dates=True)

df = pd.DataFrame(sql_query, columns=['id','common_name', 'id_bioeco','inchi', 'inchi_key', 'smiles'])

df

,id,common_name,id_bioeco,inchi,inchi_key,smiles
0,1,tobramycin,9222799514525203483,InChI=1S/C18H37N5O9/c19-3-9-8(25)2-7(22)17(29-...,NLVFBUXFDBBNBW-PBSUHMDJSA-N,C1C(C(C(C(C1N)OC2C(C(C(C(O2)CO)O)N)O)O)OC3C(CC...
1,2,Telithromycin,9222710493671371933,InChI=1S/C43H65N5O10/c1-12-33-43(8)37(48(41(53...,LJVAJPDWBABPEJ-UHFFFAOYSA-N,CCC1C2(C(C(C(=O)C(CC(C(C(C(=O)C(C(=O)O1)C)C)OC...
2,3,UNII-YCP4CS0W03,9222602442150781005,InChI=1S/C6H12N4.H2O4S/c1-4(2)10-6(8)5(7)3-9-1...,VKGPVOKDWZAKKE-UHFFFAOYSA-N,CC(C)N1C(=C(C=N1)N)N.OS(=O)(=O)O
3,4,"3,5-DIMETHYLPHENOL",9222471406831838542,"InChI=1S/C8H10O/c1-6-3-7(2)5-8(9)4-6/h3-5,9H,1...",TUAMRELNJMMDMT-UHFFFAOYSA-N,CC1=CC(=CC(=C1)O)C
4,5,5-Oxooctanoic acid,9222286964594510425,InChI=1S/C8H14O3/c1-2-4-7(9)5-3-6-8(10)11/h2-6...,OHGVWVVPDOAZSH-UHFFFAOYSA-N,CCCC(=O)CCCC(=O)O
5,6,AC1LCVJ0,9222148300385709759,InChI=1S/C10H14N5O7P/c11-8-5-9(13-2-12-8)15(3-...,UDMBCSSLTHHNCD-ASCDQALMSA-N,C1=NC2=C(C(=N1)N)N=CN2C3C(C(C(O3)COP(=O)(O)O)O)O
6,7,3-Chloro-5-fluoroisonicotinic acid,9221885079977129855,InChI=1S/C6H3ClFNO2/c7-3-1-9-2-4(8)5(3)6(10)11...,APKYTXKVPCYUOG-UHFFFAOYSA-N,C1=C(C(=C(C=N1)Cl)C(=O)O)F
7,8,None,9221877112145259159,InChI=1S/C61H86N13O13P.CN.Co/c1-29-21-38-39(22...,WNHNILJCIPJCDB-UHFFFAOYSA-N,CC1=CC2=C(C=C1C)N(C=N2)C3C(C(C(O3)CO)O[P+](=O)...
8,9,2-(Hexyloxy)ethanol,9221440159283858055,"InChI=1S/C8H18O2/c1-2-3-4-5-7-10-8-6-9/h9H,2-8...",UPGSWASWQBLSKZ-UHFFFAOYSA-N,CCCCCCOCCO
9,10,Methyl jasmonate,9221308421455500483,InChI=1S/C13H20O3/c1-3-4-5-6-11-10(7-8-12(11)1...,GEWDNTWNSAZUDX-PLNGDYQASA-N,CCC=CCC1C(CCC1=O)CC(=O)OC


In [6]:
# Which are the metabolites that are duplicated?

df_met_dupl = df[df.duplicated(['common_name','inchi','inchi_key', 'smiles'], keep = False)]

dupl_met = {}                                           # create a dictionary {metab:[id,id,...]}
for k,g in df_met_dupl.groupby(['common_name'])['id']:
    g = list(g)
    if k not in dupl_met:
        dupl_met[k] = g
    elif k in dupl_met:
        dupl_met[k].extend(g)
        
        
# print(df[df.common_name == 'Glycyrrhizic acid'])   
# the metab 'Glycyrrhizic acid' has 3 ids for the same metabolite but just 2 of them are equal, the other as different InChIKey

print(dupl_met)
    
                

{'4-Benzyloxyphenylacetic acid': [1159, 4062], 'AC1L2SOM': [1204, 3135], 'Creatine phosphate disodium salt': [546, 3567], 'Glycyrrhizic acid': [2592, 3708], 'HYDROXOCOBALAMIN': [2745, 3973], 'L-cystine': [679, 3367], 'Pregna-1,4-diene-3,20-dione,16,21-bis(acetyloxy)-9-fluoro-11,17-dihydroxy-, (11b,16a)-': [3004, 3776], 'SCHEMBL952350': [2945, 4008], 'Temsirolimus': [169, 3514], 'aluminium triphosphate': [3059, 3805], 'deltamethrin': [624, 2567], 'pimaricin': [2974, 4196]}


In [7]:
cnx.close()

Now using other query we'll retrieve the prices for a spcefic period of time **(01/18 a 06/19)**. This query will be the representation of the total number of prices that exist on **bioanalysis** and that have only the **mass units** (g and derivatives).

In [8]:
# Dataframe com query a selecionar todos os preços com unidades de g ou derivados, e no período de tempo entre 01/18 e 06/19

cnx = mysql.connector.connect(user='root', password='root',
                              host='127.0.0.1',
                              database='bioeco')

sql_query = pd.read_sql_query(''' select bioanalysis_metabolite_price.metabolite_id       as metab_id,  
                                  bioanalysis_metabolite.common_name                      as metab_name,
                                  bioanalysis_metabolite_price.date, 
                                  bioanalysis_metabolite_price.price                      as price, 
                                  bioanalysis_metabolite_price.amount                     as amount, 
                                  bioanalysis_metabolite_price.unity                      as unit, 
                                  if(provider.name is null, "None", provider.name)        as provider_name, 
                                  bioanalysis_metabolite_price.source                     as source,
                                  price/amount                                            as price_per_unit,
                                  price/(amount*convert_to_g(unity))                      as price_per_g,
                                  year(date)                                              as year, 
                                  month(date)                                             as month
                                  from bioanalysis_metabolite_price
                                  JOIN provider 
                                      on provider.id = bioanalysis_metabolite_price.provider_id
                                  JOIN bioanalysis_metabolite
                                      on bioanalysis_metabolite_price.metabolite_id = bioanalysis_metabolite.id
                                  where convert_to_g(unity) is not null and ((year(date)*10000) + (month(date)*100) between 20180100 and 20190600)
                              ''', cnx, index_col = 'date', parse_dates=True)  

df = pd.DataFrame(sql_query, columns=['metab_id','metab_name', 'price','amount', 'unit', 'provider_name', 'source', 'price_per_unit', 'price_per_g', 'year', 'month'])

df

,metab_id,metab_name,price,amount,unit,provider_name,source,price_per_unit,price_per_g,year,month
date,,,,,,,,,,,
2018-01-11 21:39:13,1,tobramycin,158.0,1.0,g,ACC Corporation,CHEMSPACE,158.00,158.00000,2018,1
2018-02-11 23:46:53,1,tobramycin,158.0,1.0,g,ACC Corporation,CHEMSPACE,158.00,158.00000,2018,2
2018-03-11 23:47:07,1,tobramycin,158.0,1.0,g,ACC Corporation,CHEMSPACE,158.00,158.00000,2018,3
2018-04-11 22:47:14,1,tobramycin,158.0,1.0,g,ACC Corporation,CHEMSPACE,158.00,158.00000,2018,4
2018-05-11 22:47:27,1,tobramycin,158.0,1.0,g,ACC Corporation,CHEMSPACE,158.00,158.00000,2018,5
2018-06-12 01:20:49,1,tobramycin,158.0,1.0,g,ACC Corporation,CHEMSPACE,158.00,158.00000,2018,6
2018-07-12 02:29:45,1,tobramycin,158.0,1.0,g,ACC Corporation,CHEMSPACE,158.00,158.00000,2018,7
2018-08-23 01:54:41,1,tobramycin,158.0,1.0,g,ACC Corporation,CHEMSPACE,158.00,158.00000,2018,8
2018-09-23 01:54:58,1,tobramycin,158.0,1.0,g,ACC Corporation,CHEMSPACE,158.00,158.00000,2018,9


In [9]:
# To see how many null values exist in each column
df.isnull().sum()

metab_id            0
metab_name        364
price               0
amount              0
unit                0
provider_name       0
source              0
price_per_unit    295
price_per_g       295
year                0
month               0
dtype: int64

In [10]:
df[df.amount == 0]  #there are prices that have the amount equal to zero ...

,metab_id,metab_name,price,amount,unit,provider_name,source,price_per_unit,price_per_g,year,month
date,,,,,,,,,,,
2018-10-22 15:22:54,67,D-cycloserine,105.0,0.0,g,TargetMol,CHEMSPACE,NaN,NaN,2018,10
2018-11-22 16:23:15,67,D-cycloserine,105.0,0.0,g,TargetMol,CHEMSPACE,NaN,NaN,2018,11
2018-12-07 14:41:16,67,D-cycloserine,105.0,0.0,g,TargetMol,CHEMSPACE,NaN,NaN,2018,12
2019-01-07 14:41:27,67,D-cycloserine,105.0,0.0,g,TargetMol,CHEMSPACE,NaN,NaN,2019,1
2019-02-07 14:41:35,67,D-cycloserine,105.0,0.0,g,TargetMol,CHEMSPACE,NaN,NaN,2019,2
2019-03-07 14:41:44,67,D-cycloserine,105.0,0.0,g,TargetMol,CHEMSPACE,NaN,NaN,2019,3
2019-04-07 13:41:51,67,D-cycloserine,105.0,0.0,g,TargetMol,CHEMSPACE,NaN,NaN,2019,4
2018-10-22 23:49:04,223,ERYTHRITOL,24.0,0.0,g,TargetMol,CHEMSPACE,NaN,NaN,2018,10
2018-11-23 00:50:09,223,ERYTHRITOL,24.0,0.0,g,TargetMol,CHEMSPACE,NaN,NaN,2018,11


The prices where the amount was zero made the price per unit and price per gram be null too.

In [11]:
# Create new dataframe where there isn't prices with amounts equal to 0
df_noNA = df[~(df['amount'] == 0)]

In [12]:
df_noNA.isnull().sum()

metab_id            0
metab_name        364
price               0
amount              0
unit                0
provider_name       0
source              0
price_per_unit      0
price_per_g         0
year                0
month               0
dtype: int64

New dataframe **df_noNA** doesn't have null prices. It's clean of prices with amounts equal to zero.

In [13]:
df_noNA.count()

metab_id          836593
metab_name        836229
price             836593
amount            836593
unit              836593
provider_name     836593
source            836593
price_per_unit    836593
price_per_g       836593
year              836593
month             836593
dtype: int64

Here we can see that the null values of the price per unit and price per gram columns were erased, however we can also see that the nulls from the metab_name column weren't erased, and that's the reason why there are less rows.

Nevertheless, it's not problematic.

In [14]:
# Dataframe with no NAN and no duplicates!!! 

df_clean = df_noNA.reset_index().drop_duplicates().set_index('date') # drop all duplicates including the index 
# for that we had to convert the index as a column and then put it again as index

df_clean

,metab_id,metab_name,price,amount,unit,provider_name,source,price_per_unit,price_per_g,year,month
date,,,,,,,,,,,
2018-01-11 21:39:13,1,tobramycin,158.0,1.0,g,ACC Corporation,CHEMSPACE,158.00,158.00000,2018,1
2018-02-11 23:46:53,1,tobramycin,158.0,1.0,g,ACC Corporation,CHEMSPACE,158.00,158.00000,2018,2
2018-03-11 23:47:07,1,tobramycin,158.0,1.0,g,ACC Corporation,CHEMSPACE,158.00,158.00000,2018,3
2018-04-11 22:47:14,1,tobramycin,158.0,1.0,g,ACC Corporation,CHEMSPACE,158.00,158.00000,2018,4
2018-05-11 22:47:27,1,tobramycin,158.0,1.0,g,ACC Corporation,CHEMSPACE,158.00,158.00000,2018,5
2018-06-12 01:20:49,1,tobramycin,158.0,1.0,g,ACC Corporation,CHEMSPACE,158.00,158.00000,2018,6
2018-07-12 02:29:45,1,tobramycin,158.0,1.0,g,ACC Corporation,CHEMSPACE,158.00,158.00000,2018,7
2018-08-23 01:54:41,1,tobramycin,158.0,1.0,g,ACC Corporation,CHEMSPACE,158.00,158.00000,2018,8
2018-09-23 01:54:58,1,tobramycin,158.0,1.0,g,ACC Corporation,CHEMSPACE,158.00,158.00000,2018,9


In [15]:
df_clean.isnull().sum() 

metab_id            0
metab_name        364
price               0
amount              0
unit                0
provider_name       0
source              0
price_per_unit      0
price_per_g         0
year                0
month               0
dtype: int64

In [16]:
df_clean.count()

metab_id          825408
metab_name        825044
price             825408
amount            825408
unit              825408
provider_name     825408
source            825408
price_per_unit    825408
price_per_g       825408
year              825408
month             825408
dtype: int64

Has it is possible to observe, this new dataframe **df_clean**, besides **not having NaN** values, also **doesn't have duplicates**!

This doesn't change the number of metabolites and providers, but changes the number of prices: 
**825408 preços**

In [17]:
dupl_met                                    #dictionary of duplicated metabolites

{'4-Benzyloxyphenylacetic acid': [1159, 4062],
 'AC1L2SOM': [1204, 3135],
 'Creatine phosphate disodium salt': [546, 3567],
 'Glycyrrhizic acid': [2592, 3708],
 'HYDROXOCOBALAMIN': [2745, 3973],
 'L-cystine': [679, 3367],
 'Pregna-1,4-diene-3,20-dione,16,21-bis(acetyloxy)-9-fluoro-11,17-dihydroxy-, (11b,16a)-': [3004,
  3776],
 'SCHEMBL952350': [2945, 4008],
 'Temsirolimus': [169, 3514],
 'aluminium triphosphate': [3059, 3805],
 'deltamethrin': [624, 2567],
 'pimaricin': [2974, 4196]}

In [18]:
# Change prices of duplicated metabolites so they are set to only one metabolite!

# df_clean1 = df_clean.copy()                                                   # to keep df_clean from being modified
 
for name in dupl_met:                                                         # for each name of the metabolite in dictionary of the duplicates
    id_old = dupl_met[name][1]                                                # save as id_old the id that its gonna change (preferably the more recent id)
    id_new = dupl_met[name][0]                                                # save as id_new the id for which the id_old is gonna change (preferably the older one)
    if name in df_clean.metab_name.unique():                                 # if that metabolite name exists in the dataframe
        df_test = df_clean[df_clean.metab_name == name]                     # create a new df just with the prices for that metabolite name
#         print(df_test)
#         print(name, df_test.metab_id.unique(), id_new)
        if id_new in df_test.metab_id.unique():                               # if the new id exists as an id (this makes sure that if the id does not exist the old one doesn't need to be changed)
#             print(name, df_test.metab_id.unique(), id_new)
            df_clean.loc[df_clean.metab_id == id_old, 'metab_id'] = id_new  # then, change the old one for the new id
#     print(df_clean1[df_clean.metab_name == name])

  
# print('Number of Metabolites with duplicates: ', df_clean.metab_id.nunique())

print('Number of Metabolites without duplicates: ', df_clean.metab_id.nunique())

2278

In [19]:
df_clean.provider_name.nunique()

126

The dataframe **df_clean** is now update **without duplicated metabolites**!

It has now 2278 metabolites, 126 providers and 825408 prices.

In [20]:
# as we can see there is only mass units in this dataframe
df_clean['unit'].unique()

array(['g', 'mg', 'kg'], dtype=object)

- **How many providers have prices for all the months??**
**And for 90% of the months?** (regardless of the metabolite)

And what about the providers?? Are they consistently lacking prices for several months independently of the metabolite?

In [21]:
# Calculating how many metabolites have prices for **all** the months of the time window

dict_dates = {}                          # Create a dictionary that will have {metab_id : [(year,month)]}
t_window = []                            # Create a list of the months that include the time window
for k,g in df_clean.groupby(['metab_id','year','month'])['price_per_g']:  # group by metabolite id, year and month 
    k = list(k)                          # Turn to a list so the output is [metab_id, year, month]
#     print(len(g))
    num_p = len(g)                       # setting number of prices for each date and each met_id (for the future maybe)
    date = (k[1],k[2])                   # turn into a tuple as (year, month)
    if date not in t_window:
        t_window.append(date)            
    if k[0] not in dict_dates:
        dict_dates[k[0]] = [date]
    elif k[0] in dict_dates:
        dict_dates[k[0]].append(date)

        
# print(dict_dates)
# print(t_window)
len_met = len(dict_dates)


num_met_all_month = 0                    # count the number of metabolites that have prices for all the months
met_all_month = []                       # list of the metabolites ids of those metabolites mentioned above
met_not_all_m = []                       # list of the metabolites ids that don't have prices for all the months
for id in dict_dates:                    
    if dict_dates[id] == t_window:       # comparing the two lists to see if they are equal!
        met_all_month.append(id)
        num_met_all_month += 1
    else: met_not_all_m.append(id)
        
               
print('Number of metabolites that have prices for all the months:', num_met_all_month, '\n')
# print('Metabolite Ids for the ones that have prices for all the months: \n', met_all_month, '\n')
print('Percentage:', num_met_all_month*100/len_met,'%', '(',num_met_all_month,'of',len_met,'metabolites) \n')


Number of metabolites that have prices for all the months: 506 

Percentage: 22.212467076382794 % ( 506 of 2278 metabolites) 



In [22]:
# Making sure there is no errors
dict_dates[met_not_all_m[0]]   # it really is lacking the first month of 2018

[(2018, 2),
 (2018, 3),
 (2018, 4),
 (2018, 5),
 (2018, 6),
 (2018, 7),
 (2018, 8),
 (2018, 9),
 (2018, 10),
 (2018, 11),
 (2018, 12),
 (2019, 1),
 (2019, 2),
 (2019, 3),
 (2019, 4),
 (2019, 5),
 (2019, 6)]

In [23]:
# And for 90% of the months? 

num_met_90_month = 0                     # count the number of metabolites that have prices for 90% of the months
met_90_month = []                        # list of the metabolites ids of those metabolites mentioned above
met_not_90_m = []                        # list of the metabolites ids that don't have prices for 90% of the months
len_window = len(t_window)               # there are 18 months so the len will be 18
for id in dict_dates:
    len_id = len(dict_dates[id])        
    if all(date in t_window for date in dict_dates[id]):   # even if the metabolite doesn't have all the months they need to be contained on the t_window list
        if len_id/len_window >= 0.9:     # find the metabolites that have prices for 90% of the months by comparing the lens of the lists  
            met_90_month.append(id)
            num_met_90_month += 1
        else: met_not_90_m.append(id)
            
len_met = len(dict_dates)

        
print('Number of metabolites that have prices for 90% of the months:', num_met_90_month, '\n')
# print('Metabolite Ids of the ones that have prices for 90% of the months: \n', met_90_month, '\n')
print('Percentage:', num_met_90_month*100/len_met,'%', '(', num_met_90_month,'of',len_met,'metabolites) \n')    

Number of metabolites that have prices for 90% of the months: 1361 

Percentage: 59.74539069359087 % ( 1361 of 2278 metabolites) 



In [24]:
met_90_month

[1,
 4,
 5,
 7,
 9,
 12,
 13,
 17,
 23,
 26,
 28,
 30,
 31,
 32,
 34,
 36,
 39,
 41,
 42,
 46,
 48,
 49,
 55,
 56,
 57,
 61,
 63,
 67,
 68,
 70,
 73,
 74,
 76,
 77,
 79,
 83,
 86,
 88,
 89,
 90,
 94,
 95,
 96,
 97,
 103,
 104,
 118,
 121,
 125,
 129,
 131,
 133,
 135,
 138,
 140,
 147,
 149,
 150,
 153,
 154,
 155,
 160,
 171,
 172,
 173,
 175,
 176,
 177,
 183,
 185,
 191,
 194,
 197,
 201,
 202,
 203,
 205,
 209,
 212,
 217,
 218,
 223,
 225,
 229,
 236,
 237,
 241,
 248,
 251,
 252,
 264,
 265,
 267,
 268,
 273,
 274,
 275,
 276,
 278,
 280,
 282,
 290,
 292,
 293,
 298,
 300,
 307,
 310,
 317,
 319,
 321,
 322,
 323,
 328,
 331,
 334,
 340,
 348,
 350,
 355,
 356,
 358,
 360,
 363,
 364,
 365,
 371,
 373,
 374,
 376,
 377,
 379,
 382,
 387,
 396,
 397,
 398,
 408,
 411,
 412,
 415,
 420,
 421,
 426,
 427,
 428,
 429,
 430,
 431,
 433,
 435,
 439,
 440,
 447,
 453,
 460,
 462,
 468,
 473,
 474,
 476,
 481,
 485,
 491,
 497,
 502,
 507,
 508,
 510,
 517,
 523,
 524,
 530,
 532,
 533,

In [25]:
# Calculating how many providers have prices for **all** the months of the time window

dict_dates_prov = {}                     # Create a dictionary that will have {prov_name : [(year,month)]}
t_window = []                            # Create a list of the months that include the time window
for k,g in df_clean.groupby(['provider_name','year','month'])['price_per_g']:  # group by provider_name, year and month 
    k = list(k)                          # Turn to a list so the output is [prov_name, year, month]
#     print(k)
#     print(g)
    num_p = len(g)                       # setting number of prices for each date and each prov_name (for the future maybe)
    date = (k[1],k[2])                   # turn into a tuple as (year, month)   
    if date not in t_window:
        t_window.append(date)            # put on list t_window all months with prices (in other words, all months of the time window)
    if k[0] not in dict_dates_prov:
        dict_dates_prov[k[0]] = [date]
    elif k[0] in dict_dates_prov:
        dict_dates_prov[k[0]].append(date)

        
num_all_providers = len(dict_dates_prov)
# print(dict_dates_prov)

num_prov_all_month = 0                    # count the number of providers that have prices for all the months
prov_all_month = []                       # list of the providers names of those metabolites mentioned above
prov_not_all_m = []                       # list of the providers names that don't have prices for all the months
for name in dict_dates_prov:                    
    if dict_dates_prov[name] == t_window:       # comparing the two lists to see if they are equal!
        prov_all_month.append(name)
        num_prov_all_month += 1
    else: prov_not_all_m.append(name)
        
               
print('Number of providers that have prices for all the months:', num_prov_all_month, '\n')
print('Providers names of the ones that have prices for all the months: \n', prov_all_month, '\n')
print('Percentage:', num_prov_all_month*100/num_all_providers,'%', '(', num_prov_all_month,'of',num_all_providers,'providers) \n')
print('-----------------------------------------------------')
print('Providers names of the ones that don\'t have prices for all months:\n', prov_not_all_m,'\n')

Number of providers that have prices for all the months: 0 

Providers names of the ones that have prices for all the months: 
 [] 

Percentage: 0.0 % ( 0 of 126 providers) 

-----------------------------------------------------
Providers names of the ones that don't have prices for all months:
 ['A1 BioChem Labs', 'AA BLOCKS', 'ACB Blocks', 'ACC Corporation', 'ACINTS', 'ACS Scientific Inc', 'AD Chemical Inc.', 'AK Scientific', 'AK Scientific Inc.', 'AK Scientific, Inc.', 'AKL Research LLP', 'AOBChem USA', 'AbaChemscene', 'Abamachem Ltd', 'Acadechem Co., Ltd', 'Accela ChemBio Inc.', 'Advamacs', 'Aldlab', 'Aldlab Chemicals LLC', 'Alichem', 'Alinda Balt', 'Alinda Chemical Trading Company', 'Alinda Chemical, Ltd.', 'Ambeed, Inc.', 'AnalytiCon Discovery, GmbH', 'Angel Pharmatech Ltd.', 'Angene', 'Angene (China)', 'Angene International Limited', 'Apollo Scientific', 'Arctom Chemicals LLC', 'Ark Pharm, Inc.', 'Aronis', 'Astatech Inc', 'Axon Medchem', 'BCH Research LLP', 'BIONET - Key Organic

In [26]:
# Making sure there is no errors
dict_dates_prov[prov_not_all_m[0]]   

[(2018, 11),
 (2018, 12),
 (2019, 1),
 (2019, 2),
 (2019, 3),
 (2019, 4),
 (2019, 5),
 (2019, 6)]

In [27]:
# And for 90% of the months? 

num_prov_90_month = 0                     # count the number of providers that have prices for at least 90% of the months
prov_90_month = []                        # list of the providers names of those providers mentioned above
prov_not_90_m = []                        # list of the providers names that don't have prices for 90% of the months
len_window = len(t_window)               # there are 18 months so the len will be 18
for name in dict_dates_prov:
    len_name = len(dict_dates_prov[name])        
    if all(date in t_window for date in dict_dates_prov[name]):   # even if the providers doesn't have all the months they need to be contained on the t_window list
        if len_name/len_window >= 0.9:     # find the providers that have prices for 90% of the months by comparing the lens of the lists  
            prov_90_month.append(name)
            num_prov_90_month += 1
        else: prov_not_90_m.append(name)

        
print('Number of providers that have prices for 90% of the months:', num_prov_90_month, '\n')
print('Providers names of the ones that have prices for 90% of the months: \n', prov_90_month, '\n')
print('Percentage:', num_prov_90_month*100/num_all_providers,'%', '(', num_prov_90_month,'of',num_all_providers,'providers) \n')        
print('-----------------------------------------------------')
print('Providers names of the ones that don\'t have prices for 90% of the months:\n', prov_not_90_m,'\n')

Number of providers that have prices for 90% of the months: 72 

Providers names of the ones that have prices for 90% of the months: 
 ['ACINTS', 'ACS Scientific Inc', 'AK Scientific', 'AK Scientific Inc.', 'AK Scientific, Inc.', 'AKL Research LLP', 'AOBChem USA', 'Abamachem Ltd', 'Accela ChemBio Inc.', 'Advamacs', 'Aldlab', 'Alichem', 'Alinda Balt', 'AnalytiCon Discovery, GmbH', 'Angene', 'Angene International Limited', 'Apollo Scientific', 'Aronis', 'Axon Medchem', 'BIOTREND Chemicals, AG', 'BioBlocks', 'Biosynth AG', 'Cayman Europe', 'Chem4Pharma', 'ChemBridge Corp.', 'ChemBridge Corporation', 'ChemDiv, Inc.', 'ChemFaces', 'ChemShuttle', 'Curpys Chemicals', 'Debye Scientific', 'EDASA Scientific', 'ENAMINE Ltd.', 'Enamine Ltd Chemspace partner', 'EvoBlocks, Ltd.', 'Eximed', 'FCH Group', 'FCH Group Chemspace partner', 'Fluorochem', 'Fluorochem Limited', 'HTS Biochemie Innovationen', 'Intermed Ltd', 'Key Organics, Ltd.', 'Labseeker', 'LeadGen Labs', 'Life Chemicals Inc.', 'Manchester O

In [28]:
# Making sure there is no errors
dict_dates_prov[prov_not_90_m[0]]   

[(2018, 11),
 (2018, 12),
 (2019, 1),
 (2019, 2),
 (2019, 3),
 (2019, 4),
 (2019, 5),
 (2019, 6)]

**Question:** What do we do with the providers that don't have prices for 90% of the months consistently? Do we filter them out?

Since we can only make an analysis per metabolite, I choose the **metabolite 67** has study case, since it's a metabolite that is in the category of the metabolites that do not have prices for all months. 

Furthermore, this metabolite does not have duplicates.

## Study Case - Metabolite and price analysis

In [29]:
# Manually choose the metab_id to analyse
# x_id = 1175
# df_studCase = df_clean[df_clean.metab_id == x_id]     #filter the dataframe where the metabolite id is equal to the study case

In [30]:
# Randomly choose the metab_id to analyse

num = np.random.randint(1,2278)

df_studCase = df_clean.copy()
if num in df_clean.metab_id.unique():
    print('Metab_id:', num)
    df_studCase = df_clean[df_clean.metab_id == num]     #filter the dataframe where the metabolite id is equal to the study case
else: 
    print('This metab_id does not exist!')
    df_studCase = None

Metab_id: 48


In [31]:
df_studCase

,metab_id,metab_name,price,amount,unit,provider_name,source,price_per_unit,price_per_g,year,month
date,,,,,,,,,,,
2018-02-08 19:54:01,48,CYCLOHEXANOL,280.00,10.0,g,"CARBONE SCIENTIFIC CO., LTD",CHEMSPACE,28.000000,28.000000,2018,2
2018-03-08 20:42:46,48,CYCLOHEXANOL,280.00,10.0,g,"CARBONE SCIENTIFIC CO., LTD",CHEMSPACE,28.000000,28.000000,2018,3
2018-04-08 19:56:00,48,CYCLOHEXANOL,280.00,10.0,g,"CARBONE SCIENTIFIC CO., LTD",CHEMSPACE,28.000000,28.000000,2018,4
2018-05-08 23:58:07,48,CYCLOHEXANOL,280.00,10.0,g,"CARBONE SCIENTIFIC CO., LTD",CHEMSPACE,28.000000,28.000000,2018,5
2018-06-09 12:36:23,48,CYCLOHEXANOL,280.00,10.0,g,"CARBONE SCIENTIFIC CO., LTD",CHEMSPACE,28.000000,28.000000,2018,6
2018-07-09 12:36:54,48,CYCLOHEXANOL,280.00,10.0,g,"CARBONE SCIENTIFIC CO., LTD",CHEMSPACE,28.000000,28.000000,2018,7
2018-08-10 13:32:46,48,CYCLOHEXANOL,280.00,10.0,g,"CARBONE SCIENTIFIC CO., LTD",CHEMSPACE,28.000000,28.000000,2018,8
2018-09-10 13:33:49,48,CYCLOHEXANOL,280.00,10.0,g,"CARBONE SCIENTIFIC CO., LTD",CHEMSPACE,28.000000,28.000000,2018,9
2018-10-10 13:33:52,48,CYCLOHEXANOL,280.00,10.0,g,"CARBONE SCIENTIFIC CO., LTD",CHEMSPACE,28.000000,28.000000,2018,10


In [32]:
# Number of prices for metabolite X
df_studCase.shape[0]

1071

In [33]:
# Verify if dataframe as only mass units
df_studCase.unit.unique()

array(['g', 'mg'], dtype=object)

In [34]:
# How many different providers this metabolite has?
df_studCase.provider_name.nunique()

27

In [35]:
# How many different sources this metabolite has? 
df_studCase.source.unique()

array(['CHEMSPACE', 'MolPort', 'OXChem'], dtype=object)

This metabolite, (id = 1166), has **5662 prices**, counting already only the ones with **mass units** and **without duplicated prices**. Besides, this metabolite has **39 providers** and only **3 sources** (which may be good).

#### Boxplot of prices por month 

In [36]:
for k,g in df_studCase.groupby(['year','month'])['price_per_g']:
    print(list(k))
    print(g)

[2018, 2]
date
2018-02-08 19:54:01    28.000000
2018-02-08 19:54:00     0.000116
2018-02-08 19:54:00     0.000072
2018-02-08 19:54:00    41.000000
2018-02-08 19:54:00    33.600000
                         ...    
2018-02-08 19:54:00     0.004802
2018-02-08 19:54:00     0.004205
2018-02-08 19:54:00     0.003846
2018-02-08 19:54:00     0.002892
2018-02-08 19:54:00     0.002414
Name: price_per_g, Length: 79, dtype: float64
[2018, 3]
date
2018-03-08 20:42:46    28.000000
2018-03-08 20:42:45     0.000116
2018-03-08 20:42:45     0.000072
2018-03-08 20:42:45    41.000000
2018-03-08 20:42:45    33.600000
                         ...    
2018-03-08 20:42:45     0.004802
2018-03-08 20:42:45     0.004205
2018-03-08 20:42:45     0.003846
2018-03-08 20:42:45     0.002892
2018-03-08 20:42:45     0.002414
Name: price_per_g, Length: 83, dtype: float64
[2018, 4]
date
2018-04-08 19:56:00    28.000000
2018-04-08 19:55:58     0.000116
2018-04-08 19:55:58     0.000072
2018-04-08 19:55:58    41.000000
2018-

In [37]:
# Create boxplot with prices per month
# Create plot with plotly

df_studCase = df_studCase.sort_index()


layout = go.Layout(
    title = 'Boxplot of Methane Prices per Month',
    yaxis = dict(
        title = 'Price (USD)'
    ),
    xaxis = dict(
        title = 'Month'
    )
)


dict_month = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Ago', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}
    
month_list = []
d_y = {}
for k,g in df_studCase.groupby(['year','month'])['price_per_g']:
    k = list(k)
    k[1] = dict_month[k[1]]
    k = k[1] +' '+ str(k[0])
    month_list.append(k)
# print(month_list)
    y = g.values
    if k not in d_y:
        d_y[k] = y
    elif k in d_y:
        d_y[k].append(y)
    else: print('There is a key lacking on the y dict.')


def update_plot(signals):#, freq):
    data1 = []
    for s in signals:
        trace1 = go.Box(
            y = d_y[s],
            name =  '{}'.format(s),
            boxpoints='outliers',
            boxmean=True                                       # show mean
        )
    
        data1.append(trace1)
      
    fig = go.Figure(data = data1, layout= layout)
    po.iplot(fig)
    
# data = []
# for s in month_list:
#     trace1 = go.Box(
#         y = d_y[s],
#         name = '{}'.format(s),
#         boxpoints='outliers'
#     )

#     data.append(trace1)


signals = widgets.SelectMultiple(options = month_list, description = 'Date')
#freq = widgets.FloatSlider(min=1, max=20, value=1, description = 'Freq')
widgets.interactive(update_plot, signals = signals)#, freq = freq)


interactive(children=(SelectMultiple(description='Date', options=('Feb 2018', 'Mar 2018', 'Apr 2018', 'May 201…

Como podemos ver a variância muda muito entre os diferentes boxplots de cada mês! 

**??**  já nem sei o que é a variância....

#### Boxplot of prices por month and provider

In [64]:
# Create boxplot with prices per month and ** per provider**!!
# Create plot with plotly

df_studCase = df_studCase.sort_index()


layout = go.Layout(
    title = 'Boxplot of Methane Prices per Month and per Provider',
    yaxis = dict(
        title = 'Price (USD)'
    ),
    xaxis = dict(
        title = 'Month'
    )
)

#dicionário com os meses todos para poder substituir o número pela palavra
dict_month = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Ago', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}

#valores para a opção all_providers
all_p = []    
for k,g in df_studCase.groupby(['year','month'])['price_per_g']:     # só se agrupa por mês e ano
    k = list(k)                                                # o mês e o ano têm de ficar como lista
    k[1] = dict_month[k[1]]                                    # substituir o nº do mês pela palavra
    date = k[1] +' '+ str(k[0])                                # criar a string date que vai ficar no x axis
    y = g.values                                           
    for i in y:                                                # colocar numa lista cada valor com a respectiva data      
        all_p.append([date, i])

dict_boxplot = {} #Fazer um dicionário onde os values são listas de listas (ex., [date, valor])
          
dict_boxplot['all_providers'] = all_p                          # colocar os valores de all_providers no dicionário
for k,g in df_studCase.groupby(['year','month', 'provider_name'])['price_per_g']:  # agrupar por mês, ano e provider
    k = list(k)                                                # mês, ano e provider têm de ficar como lista para ser mais fácil usar
    k[1] = dict_month[k[1]]                                                        
    date = k[1] +' '+ str(k[0]) 
    y = g.values
    values =[]                                                 # criar lista onde vão ficar as listas 
    for i in y:
        values.append([date, i])                               # pôr na lista uma lista com um dos valores e a respectiva data
#     print(values)
    if k[2] not in dict_boxplot:
        dict_boxplot[k[2]] = values                            # colocar no dicionário o provider e a respectiva lista de listas
    elif k[2] in dict_boxplot:
        dict_boxplot[k[2]].extend(values)                       # se já existir extender a lista existente com outras listas

# print(dict_boxplot)        


list_options=[]                                                # criar uma lista que tenha todos os provviders + all_providers    
for p in dict_boxplot:                                         # para poder usar como opção
    list_options.append(p)

# list_options.sort(key=str.lower)    
        
# print(len(list_options))  #dá certinho com o número de providers que este metabolito tem + a opção de todos os providers    
    
def update_plot(signals):#, freq):
    data1 = []
    for s in signals:
        trace1 = go.Box(
            y = [values[1] for values in dict_boxplot[s]],     # criar gigante lista só com os valores de y (usar valores nas listas da lista)
            x = [values[0] for values in dict_boxplot[s]],     # criar gigante lista com as datas respectivas para cada y
            name =  '{}'.format(s),
            boxpoints= 'all',                                  # 'outliers' assim mostra só os outliers
            boxmean=True                                       # show mean
        )
    
        data1.append(trace1)
      
    fig = go.Figure(data = data1, layout= layout)
    po.iplot(fig)
    

signals = widgets.SelectMultiple(options = list_options, description = 'Date')
#freq = widgets.FloatSlider(min=1, max=20, value=1, description = 'Freq')
widgets.interactive(update_plot, signals = signals)#, freq = freq)

interactive(children=(SelectMultiple(description='Date', options=('all_providers', 'Biosynth AG', 'Manchester …

In [39]:

#dictionary with all the months to substitute the number by the word
dict_month = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Ago', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}

time_window = ['Jan 2018', 'Feb 2018', 'Mar 2018', 'Apr 2018', 'May 2018', 'Jun 2018', 'Jul 2018', 'Ago 2018', 'Sep 2018',
               'Oct 2018', 'Nov 2018', 'Dec 2018', 'Jan 2019', 'Feb 2019', 'Mar 2019', 'Apr 2019', 'May 2019', 'Jun 2019']


# list of all the months on this metabolite
time_window_m = [] 
for k,g in df_studCase.groupby(['year','month'])['price_per_g']:     # só se agrupa por mês e ano
    k = list(k)                                                # o mês e o ano têm de ficar como lista
    k[1] = dict_month[k[1]]                                    # substituir o nº do mês pela palavra
    date = k[1] +' '+ str(k[0])                                # criar a string date que vai ficar no x axis
    if date not in time_window_m:
        time_window_m.append(date)


def date_str(k):    # k is the list that is created when we do the groupby (it has 3 elements: year, month, provider_name)
    k[1] = dict_month[k[1]]                                                        
    date = k[1] +' '+ str(k[0]) 
    return date


dict_prov = {}                                          # Create dictionary with list of lists {prov:[[date,price]]}
for k,g in df_studCase.groupby(['year','month', 'provider_name'])['price_per_g']:  # group by month, year and provider
    k = list(k)                                         # month, year and provider turn into a list to be easier to use
    date = date_str(k)                                  # turn date into a string 
    y = g.values
    values =[]                                          # create list where it will be the lists  
    for i in y:
        values.append([date, i])                        # put on the list 'values' a list with the date and each price [date, price]
#     print(values)
    if k[2] not in dict_prov:
        dict_prov[k[2]] = values                        # put on the dict the provider as key and the respective list of lists as values
    elif k[2] in dict_prov:
        dict_prov[k[2]].extend(values)                  # if provider already exists, extend the existing list with other lists

# print(dict_prov)
len_all_prov = len(dict_prov)                           # number of all the providers for this metabolite
len_window_m = len(time_window_m)                       # number of all the months with prices for this metabolite
        
prov_all_month = []                                     # create list with providers that have prices for all months
prov_almost_all_m = []                                  # create list with providers that have prices for all months but all the prices are below 0.5 USD
prov_flaw = []                                          # create dictionary as {provider: num_month_without_price}
prov_flaw_below_1 = []                                  # create list with providers without prices for all months and are below 0.5 USD
for prov in dict_prov:
    month = {}                                          # create dictionary as {date:[prices]}
    for values in dict_prov[prov]:                      # each list [date, price]
        if values[0] not in month:                      # if date isn't in the dict put it as key and price in list as value      
            month[values[0]] = [values[1]]
        elif values[0] in month:                        # if date is already in dict, append the price to the list of that date 
            month[values[0]].append(values[1])
#     print(month)
    if len(month) < len_window_m:                       # if len(month) is less than the number of months in this metabolite
        if any(v >= 0.5 for m in month for v in month[m]):
            prov_flaw.append(prov)                      # and if some prices are above 0.5 USD then save it on list of provider lacking months
        elif all(v > 0 and v < 0.5 for m in month for v in month[m]): # if prices are all between 0 and 0.5 USD 
            prov_flaw_below_1.append(prov)              # the provider is kept on the list below 1USD since the prices are really low
    elif len(month) == len_window_m:                    # if they have all the months in this metabolite then verify if some prices are above 0.5 USD
        if any(v >= 0.5 for m in month for v in month[m]):
            prov_all_month.append(prov)                 # if they are then save prov on the list of all months
        elif all(v > 0 and v < 0.5 for m in month for v in month[m]): # if it has 18 months but prices are all between 0 and 0.5 USD 
            prov_almost_all_m.append(prov)              # provider it's kept on list of the almost all months, since prices are too low
                    


# Find which provider doesn't have prices for all the months and how many months are lacking
# There are 18 months on total

dic_prov_flaw = {}                                           # create a dictionary as {provider:num_months_lacking}
for prov in dict_prov:                                       # for each provider
    month = {}                                               # create a new dictionary 'month' as {date:[prices]}
    for values in dict_prov[prov]:                           # for each list of [date, price]
        if values[0] not in month:                           
            month[values[0]] = [values[1]]                   # put the date as key in 'month' and the price in a list as value
        elif values[0] in month:
            month[values[0]].append(values[1])               # if the date already exists on the 'month' just append more prices 
#     print(month)
    for t in time_window_m:                                  # for each date on the list 'time_window_m'
        if t not in month.keys():                            # if the date is not on 'month'
            times = 1                                        # create a counter 'times' equal to 1
            if prov not in dic_prov_flaw:                   
                dic_prov_flaw[prov] = times                  # set the provider as a key in the dict 'dic_prov_flaw' with the value as the counter 'times'
            elif prov in dic_prov_flaw:
                dic_prov_flaw[prov] += 1                     # if the provider already exist on 'dic_prov_flaw' add 1 to the counter on the value
              
        
        
        
print('Providers with prices for each month:\n\n', prov_all_month, '\n')     #providers que têm preços todos os meses!!
print('Percentage: ', len(prov_all_month)*100/len_all_prov,'% (', len(prov_all_month),'of',len_all_prov, ' providers) \n')
print('----------------------------------------------------------------------------------\n')
print('Providers with prices for each month but below 0.5:\n\n', prov_almost_all_m, '\n')
print('Percentage: ', len(prov_almost_all_m)*100/len_all_prov,'% (', len(prov_almost_all_m),'of', len_all_prov,' providers) \n')
print('----------------------------------------------------------------------------------\n')
print('Providers which do not have prices for all months:\n\n', prov_flaw, '\n')
print('Percentage: ', len(prov_flaw)*100/len_all_prov,'% (', len(prov_flaw),'of', len_all_prov, ' providers) \n\n')
print('----------------------------------------------------------------------------------\n')
print('Providers who don\'t have prices for all the months and prices are also consistently below 0.5:\n\n', prov_flaw_below_1,'\n')
print('Percentage: ', len(prov_flaw_below_1)*100/len_all_prov,'% (', len(prov_flaw_below_1),'of', len_all_prov,' providers) \n')
print('----------------------------------------------------------------------------------\n')
print('Which providers do not have prices for all the months and how many months are lacking: \n\n', dic_prov_flaw)

Providers with prices for each month:

 ['Biosynth AG', 'Pharmeks, Ltd.', 'UkrOrgSynthesis'] 

Percentage:  11.11111111111111 % ( 3 of 27  providers) 

----------------------------------------------------------------------------------

Providers with prices for each month but below 0.5:

 ['Manchester Organics Limited', 'Maybridge, Ltd.', 'Otava, Ltd.', 'Otava, Ltd. (2 weeks)'] 

Percentage:  14.814814814814815 % ( 4 of 27  providers) 

----------------------------------------------------------------------------------

Providers which do not have prices for all months:

 ['CARBONE SCIENTIFIC CO., LTD', 'Curpys Chemicals', 'ENAMINE Ltd.', 'FCH Group Chemspace partner', 'INTERBIOSCREEN Limited', 'InterBioScreen Ltd.', 'OXChem', 'Oxchem Corporation', 'UORSY Chemspace partner', 'Enamine Ltd Chemspace partner', 'Angene International Limited', 'InterBioScreen Doo.', 'Angene', 'AK Scientific', 'AK Scientific, Inc.', 'Vitas-M Laboratory, Ltd.'] 

Percentage:  59.25925925925926 % ( 16 of 27  pr

There are really few providers with prices for all months in these metabolite. For the providers that don't have prices for all the months we will **keep only the ones that have prices for at least 90% of the months**.

In [40]:
# Which providers of the metabolite 1755 have prices for 90% of the months

# dict_prov

In [41]:
# And for 90% of the months? # colocar media num dicionario com datas e medias

prov_90 = []                             # list of providers that have prices for 90% of the months
prov_allm = []                           # list of providers that have prices for all months
len_window_m = len(time_window_m)        # number of months on this metabolite
prov_not_90 = []                         # list of the providers names that don't have prices for 90% of the months
for prov in dict_prov:                   # this dictionary has all providers {provider:[[date, price]]}
    dates = {}                           # create dictionary where {date:[prices]}
    for d in dict_prov[prov]:
        if d[0] not in dates:
            dates[d[0]] = [d[1]]
        elif d[0] in dates:
            dates[d[0]].append(d[1])
    len_dates = len(dates)                      # find number of months with prices for that provider
    if all(date in time_window_m for date in dates):   # even if the providers don't have all the months they need to be contained on the t_window list
        if len_dates/len_window_m == 1:         # find providers that have prices for all months
            prov_allm.append(prov)
        elif len_dates/len_window_m >= 0.9:     # find the providers that have prices for 90% of the months by comparing the lens of the lists  
            prov_90.append(prov)
        else: prov_not_90.append(prov)

            
print('Providers that have prices for all months: \n', prov_allm, '\n')
print('Percentage:', len(prov_allm)*100/len(dict_prov),'%', '(', len(prov_allm),'of',len(dict_prov),'providers) \n')        
print('-----------------------------------------------------')     
print('Providers that have prices for 90% of the months: \n', prov_90, '\n')
print('Percentage:', len(prov_90)*100/len(dict_prov),'%', '(', len(prov_90),'of',len(dict_prov),'providers) \n')        
print('-----------------------------------------------------')
print('Providers that don\'t have prices for 90% of the months:\n', prov_not_90,'\n')
print('Percentage:', len(prov_not_90)*100/len(dict_prov),'%', '(', len(prov_not_90),'of',len(dict_prov),'providers) \n')

Providers that have prices for all months: 
 ['Biosynth AG', 'Manchester Organics Limited', 'Maybridge, Ltd.', 'Otava, Ltd.', 'Otava, Ltd. (2 weeks)', 'Pharmeks, Ltd.', 'UkrOrgSynthesis'] 

Percentage: 25.925925925925927 % ( 7 of 27 providers) 

-----------------------------------------------------
Providers that have prices for 90% of the months: 
 [] 

Percentage: 0.0 % ( 0 of 27 providers) 

-----------------------------------------------------
Providers that don't have prices for 90% of the months:
 ['Acadechem Co., Ltd', 'CARBONE SCIENTIFIC CO., LTD', 'Curpys Chemicals', 'ENAMINE Ltd.', 'FCH Group Chemspace partner', 'Fluorochem', 'INTERBIOSCREEN Limited', 'InterBioScreen Ltd.', 'OXChem', 'Oxchem Corporation', 'UORSY Chemspace partner', 'Enamine Ltd Chemspace partner', 'Fluorochem Limited', 'Angene International Limited', 'InterBioScreen Doo.', 'Angene', 'AK Scientific', 'AK Scientific, Inc.', 'Manchester Organics', 'Vitas-M Laboratory, Ltd.'] 

Percentage: 74.07407407407408 % ( 2

**ATENTION** Here the providers that have prices for 90% of the months don't include the providers with prices for all months (in contrary to the percentages on the initial statistics)

In [42]:
def turn_month_in_list(month):
    month = month.split()
    for number, word in dict_month.items():
        if word == month[0]:
            month[0] = number
    month[1] = int(month[1])
    return month

In [43]:
def find_previous_month(month):
    for i in range(len(time_window_m)):
        if time_window_m[i]== month:
            if i-1 < 0:
                print('There is no previous month for this one. \n')
            else: 
                prev_month = time_window_m[i-1]    
                return prev_month
    

In [44]:
def avg_prev_month(previous_month, dic_fix):
    if previous_month in dic_fix:
        avg = 0
        for date in dic_fix:                                    # for each date of this provider
            if date == previous_month:                          # see if it is equal to the previous month of the lacking one                                               
                sum = 0              
                for p in dic_fix[date]:
                    sum += p                                    # sum the prices of that previous month  
                avg = sum/len(dic_fix[date])                    # and calculate the average
        return avg        
    else: 
        previous_month = find_previous_month(previous_month)    # find previous month of the previous month
        avg = avg_prev_month(previous_month)
        

In [45]:
# Create a function to rectify one provider at a time

def rectify_prov(prov, df):
    
    # Discover which month is lacking to the provider that needs to be rectified
    
    time_fix = []                                             # create list with the dates that will have to be amend
    dic_fix = {}                                              # dictionary with {date:[prices]} for that only provider
    time = []                                                 # create list with all the dates of this provider
    for value in dict_prov[prov]:                             # lists of [date, price]
        if value[0] not in dic_fix:                           # create dic_fix
            dic_fix[value[0]] = [value[1]]
        elif value[0] in dic_fix:
            dic_fix[value[0]].append(value[1])
    for date_m in dic_fix:                                    # for each time in the dictionary with the dates 
        if date_m not in time:
            time.append(date_m)
    for t in time_window_m:                                   # for each date in the time window 
        if t not in time:
            time_fix.append(t)

#     print('Months lacking prices:', time_fix)


    # Rectify the provider!

    avg_month = {}                                                  # create dictionary as {month:avg}
    for time in time_fix:                                           # for each date that needes to be fixed
        previous_month = find_previous_month(time)                  # find previous month of the lacking one
        if previous_month:
            avg = avg_prev_month(previous_month, dic_fix)
            if time not in avg_month:                                   
                avg_month[time] = avg                               # set time as a key on 'avg_month' and the average as value 
            elif time in avg_month:
                print('Atention: There is more than 1 average for a month. \n')
        else: 
            print('It\'s not possible to rectify the provider', prov, '\n\n')
            break
    
    if avg_month:
        print('Average of the previous month:', avg_month, '\n')
    
    #**These code needs to be optimizes so it can be used if the provider is lacking more than one month or various months in a row!**

    for month in avg_month:
        previous_month = find_previous_month(month)            # find previous month of 'month' 
        month = turn_month_in_list(month)                      # turn 'Month Year' into [month_number, year]    
        p_month = turn_month_in_list(previous_month)           
        last_index = df[(df['year'] == month[1]) &
                        (df['month'] == month[0])].index.max() # save the last index of the df for that month and year
        
        last_index = last_index + datetime.timedelta(minutes=30)         # add thirty minutes to the last_index to create a new index

        df_fix = df[(df.provider_name == prov) &
                    (df['year'] == p_month[1]) &
                    (df['month'] == p_month[0])].tail(1)         # save the last row of the df for that provider in the previous month
        
#         print(df_fix)
        
        metab_id, metab_name, provider_name, source = df_fix.loc[df_fix.index.max(),
                                                                 ['metab_id','metab_name',
                                                                  'provider_name','source']
                                                                ]
        # set the metab_id, metab_name, provider_name and source as the last values for that provider in the previous 
        # month so we can use them below
        
        amount = 1                                               # set amount and unit always as 1g so it doesn't mess up on
        unit = 'g'                                                 # the price_per_unit and price_per_g
        
        df_fix= pd.DataFrame([[metab_id,metab_name, avg, amount, unit, provider_name,source, avg, avg, month[1], month[0]]],
                             columns=list(df.columns), index = [last_index])
        # create a new dataframe with just one row with all the right values and the average 
        
#         print(df_fix)

        df = df.append(df_fix.loc[:])                          # add this row to the dataframe
        print(prov, 'fixed! \n\n')
        return df

In [46]:
for prov in prov_90:
    df_studCase= rectify_prov(prov, df_studCase)
    
df_studCase

,metab_id,metab_name,price,amount,unit,provider_name,source,price_per_unit,price_per_g,year,month
date,,,,,,,,,,,
2018-02-08 19:54:00,48,CYCLOHEXANOL,241.43,100.00,mg,"Maybridge, Ltd.",MolPort,2.414300,0.002414,2018,2
2018-02-08 19:54:00,48,CYCLOHEXANOL,194.00,5.00,g,UkrOrgSynthesis,MolPort,38.800000,38.800000,2018,2
2018-02-08 19:54:00,48,CYCLOHEXANOL,129.00,2.50,g,UkrOrgSynthesis,MolPort,51.600000,51.600000,2018,2
2018-02-08 19:54:00,48,CYCLOHEXANOL,70.00,1.00,g,UkrOrgSynthesis,MolPort,70.000000,70.000000,2018,2
2018-02-08 19:54:00,48,CYCLOHEXANOL,25.00,1.00,g,"Pharmeks, Ltd.",MolPort,25.000000,25.000000,2018,2
2018-02-08 19:54:00,48,CYCLOHEXANOL,65.00,5.00,g,"Pharmeks, Ltd.",MolPort,13.000000,13.000000,2018,2
2018-02-08 19:54:00,48,CYCLOHEXANOL,100.00,10.00,g,"Pharmeks, Ltd.",MolPort,10.000000,10.000000,2018,2
2018-02-08 19:54:00,48,CYCLOHEXANOL,66.88,5.00,mg,"Maybridge, Ltd.",MolPort,13.376000,0.013376,2018,2
2018-02-08 19:54:00,48,CYCLOHEXANOL,216.87,75.00,mg,"Maybridge, Ltd.",MolPort,2.891600,0.002892,2018,2


**QUESTION:** Should we rectify the first month that appears in the metabolite in question? Since it does not have a previous month should it retrieve the average price from the following month?

**ATENCAO: Este código em cima precisa URGENTEMENTE de ser optimizado para poder ser utilizado quase haja mais providers com falhas em meses, haja mais meses em falta e/ou os meses em falta estejam seguidos**

Está tudo resolvido e testado, no entanto não deu para testar a última situação uma vez que no máximo dos máximos há 18 meses e para ter preços em 90% não pode ter em falta mais de 1.8 meses. Ou seja, nunca chega a 2 meses, por isso não dá para ver que acontece caso estejam seguidos. 

In [47]:
# df_studCase.index.max()                  # watch if the max index changed as it should

In [48]:
# Clean the providers that don't have prices for at least 90% of the months 

df_studCase = df_studCase[~df_studCase.provider_name.isin(prov_not_90)] # save in the database only the data related to the providers that 
                                                            # have prices for at least 90% of the months
    
df_studCase

,metab_id,metab_name,price,amount,unit,provider_name,source,price_per_unit,price_per_g,year,month
date,,,,,,,,,,,
2018-02-08 19:54:00,48,CYCLOHEXANOL,241.43,100.0,mg,"Maybridge, Ltd.",MolPort,2.414300,0.002414,2018,2
2018-02-08 19:54:00,48,CYCLOHEXANOL,194.00,5.0,g,UkrOrgSynthesis,MolPort,38.800000,38.800000,2018,2
2018-02-08 19:54:00,48,CYCLOHEXANOL,129.00,2.5,g,UkrOrgSynthesis,MolPort,51.600000,51.600000,2018,2
2018-02-08 19:54:00,48,CYCLOHEXANOL,70.00,1.0,g,UkrOrgSynthesis,MolPort,70.000000,70.000000,2018,2
2018-02-08 19:54:00,48,CYCLOHEXANOL,25.00,1.0,g,"Pharmeks, Ltd.",MolPort,25.000000,25.000000,2018,2
2018-02-08 19:54:00,48,CYCLOHEXANOL,65.00,5.0,g,"Pharmeks, Ltd.",MolPort,13.000000,13.000000,2018,2
2018-02-08 19:54:00,48,CYCLOHEXANOL,100.00,10.0,g,"Pharmeks, Ltd.",MolPort,10.000000,10.000000,2018,2
2018-02-08 19:54:00,48,CYCLOHEXANOL,66.88,5.0,mg,"Maybridge, Ltd.",MolPort,13.376000,0.013376,2018,2
2018-02-08 19:54:00,48,CYCLOHEXANOL,216.87,75.0,mg,"Maybridge, Ltd.",MolPort,2.891600,0.002892,2018,2


### Compare providers with the average of all providers

In [49]:
dict_boxplot

{'all_providers': [['Feb 2018', 0.0024143],
  ['Feb 2018', 38.8],
  ['Feb 2018', 51.6],
  ['Feb 2018', 70.0],
  ['Feb 2018', 25.0],
  ['Feb 2018', 13.0],
  ['Feb 2018', 10.0],
  ['Feb 2018', 0.013375999999999999],
  ['Feb 2018', 0.0028916],
  ['Feb 2018', 0.01672],
  ['Feb 2018', 0.022293333333333335],
  ['Feb 2018', 23.7],
  ['Feb 2018', 33.6],
  ['Feb 2018', 44.4],
  ['Feb 2018', 52.0],
  ['Feb 2018', 92.0],
  ['Feb 2018', 140.0],
  ['Feb 2018', 0.00011600000000000001],
  ['Feb 2018', 7.2e-05],
  ['Feb 2018', 41.0],
  ['Feb 2018', 33.6],
  ['Feb 2018', 31.8],
  ['Feb 2018', 28.0],
  ['Feb 2018', 280.0],
  ['Feb 2018', 0.0005200000000000001],
  ['Feb 2018', 0.03344],
  ['Feb 2018', 29.5],
  ['Feb 2018', 0.06688],
  ['Feb 2018', 3e-05],
  ['Feb 2018', 30.0],
  ['Feb 2018', 0.0038462],
  ['Feb 2018', 0.004204500000000001],
  ['Feb 2018', 0.00011],
  ['Feb 2018', 6e-05],
  ['Feb 2018', 0.1898],
  ['Feb 2018', 0.0584],
  ['Feb 2018', 2e-05],
  ['Feb 2018', 0.004801666666666667],
  ['Feb 2

In [65]:
time_window

['Jan 2018',
 'Feb 2018',
 'Mar 2018',
 'Apr 2018',
 'May 2018',
 'Jun 2018',
 'Jul 2018',
 'Ago 2018',
 'Sep 2018',
 'Oct 2018',
 'Nov 2018',
 'Dec 2018',
 'Jan 2019',
 'Feb 2019',
 'Mar 2019',
 'Apr 2019',
 'May 2019',
 'Jun 2019']

In [66]:
# Calculating averages for each date in each provider and for all providers

avg_all_prov = []                                 # create list with tuples [(date, avg)] for the prices of all_providers
dict_avg = {}                                     # create dict with prov as key and list of tuples as value {prov:[(date, avg)]}
for prov in dict_boxplot:                         # for each provider
#     print(dict_boxplot[prov])
    for date in time_window:                      # for each date in time window
        sum = 0                                
        count_prices = 0
        avg = 0
        for value in dict_boxplot[prov]:             # for each value, in other words, each list(date, price)       
            if value[0] == date:                     # see if date of this provider equals to date of time window                                                 
                count_prices += 1                      # if so count the number of prices for these date 
                sum += value[1]                        # sum the prices of the date  
    #         print(sum)
    #         print(count_prices)
    #         print(avg)
            else: None                              # if not, don't do anything
        
        if sum:                                     # if sum exists, if it's different to zero
            avg = sum/count_prices                    # calculate the average in this date
            if prov == 'all_providers':               # if provider is all providers append the date and the avg to the related list
                avg_all_prov.append((date, avg)) 
            else:                                     # if not
                if prov not in dict_avg:                # and if the provider isn't in the dictionary
                    dict_avg[prov] = [(date, avg)]        # append to that provider the list with the tuple (date, avg)
                elif prov in dict_avg:
                    dict_avg[prov].append((date, avg))    # if it exists, append the tuple to the list
#         else: print('For the date', date,'in provider', prov, 'there is no prices.')

dict_avg

{'Biosynth AG': [('Feb 2018', 31.8),
  ('Mar 2018', 31.8),
  ('Apr 2018', 31.8),
  ('May 2018', 31.8),
  ('Jun 2018', 31.8),
  ('Jul 2018', 31.8),
  ('Ago 2018', 31.8),
  ('Sep 2018', 31.8),
  ('Oct 2018', 31.8),
  ('Nov 2018', 31.8),
  ('Dec 2018', 31.8),
  ('Jan 2019', 31.8),
  ('Feb 2019', 31.8),
  ('Mar 2019', 31.8),
  ('Apr 2019', 31.8),
  ('May 2019', 31.8),
  ('Jun 2019', 31.8)],
 'Manchester Organics Limited': [('Feb 2018', 0.021156),
  ('Mar 2018', 0.021156),
  ('Apr 2018', 0.021156),
  ('May 2018', 0.021156),
  ('Jun 2018', 0.021156),
  ('Jul 2018', 0.021156),
  ('Ago 2018', 0.021156),
  ('Sep 2018', 0.021156),
  ('Oct 2018', 0.021156),
  ('Nov 2018', 0.021156),
  ('Dec 2018', 0.021156),
  ('Jan 2019', 0.021156),
  ('Feb 2019', 0.021156),
  ('Mar 2019', 0.021156),
  ('Apr 2019', 0.021156),
  ('May 2019', 0.021156),
  ('Jun 2019', 0.021156)],
 'Maybridge, Ltd.': [('Feb 2018', 0.014198840476190477),
  ('Mar 2018', 0.014198840476190475),
  ('Apr 2018', 0.014198840476190477),
  (

In [67]:
# compare the prices: which providers are above the average of all provider and which are below

prov_above_avg = []                                      # create a list for all the providers above the average
prov_below_avg = []                                      # create a list for all the providers below the average
prov_equal_avg = []                                      # create a list for all the providers equal to the average
for prov in dict_avg:                                    # for each provider in the dict_avg
    date_above_or_below = {'above':[],'equal':[], 'below':[]}   # create a dictionary {'above': [date], 'equal': [date], 'below':[date]}
    num_dates = len(dict_avg[prov])                      # calculate the number of dates in this provider                    
    for value_prov in dict_avg[prov]:                    # for each tuple in the list of that provider, in other words, the tuple (date,avg)
        for value_all in avg_all_prov:                   # for each (date, avg) of the list of all providers
            if value_all[0] == value_prov[0]:            # if the dates are the same 
                if value_prov[1] > value_all[1]:         # if the average of the provider is higher than the total avg
                    date_above_or_below['above'].append(value_prov[0])  #then the date is appended in the 'above' on dict
                elif value_prov[1] == value_all[1]:      # if the avg of the provider is equal to the total avg
                    date_above_or_below['equal'].append(value_prov[0])  #then the date is appended in the 'equal' on dict
                elif value_prov[1] < value_all[1]:       # if the average of the provider is lower than the total avg
                    date_above_or_below['below'].append(value_prov[0])  #then the date is appended in the 'below' on dict
#     print(prov, date_above_or_below)
#     print(prov, num_dates)
#     print(prov,':', len(date_above_or_below['above']))
    if len(date_above_or_below['above'])/num_dates >= 0.9:   # if the ratio of *above* dates is higher or equal to 0.9
        prov_above_avg.append(prov)                            # then provider is considered above avg
    elif len(date_above_or_below['equal'])/num_dates >= 0.9: # if the ratio of *equal* dates is higher or equal to 0.9
        prov_equal_avg.append(prov)                            # then provider is considered equal to avg
    elif len(date_above_or_below['below'])/num_dates >= 0.9: # if the ratio of *below* dates is higher or equal to 0.9
        prov_below_avg.append(prov)                            # then provider is considered below avg
        
        
print('Providers that have at least 90% of the dates with averages **above** the averages of all providers: \n',prov_above_avg,'\n')
print('Number of providers consistently **above** all providers: ', len(prov_above_avg), '\n')
print('----------------------------------------------------------------------- \n\n')
print('Providers that have at least 90% of the dates with averages **equal** the averages of all providers: \n',prov_equal_avg,'\n')
print('Number of providers consistently **equal** all providers: ', len(prov_equal_avg), '\n')
print('----------------------------------------------------------------------- \n\n')
print('Providers that have at least 90% of the dates with averages **below** the averages of all providers: \n',prov_below_avg,'\n')
print('Number of providers consistently **below** all providers: ', len(prov_below_avg), '\n')


Providers that have at least 90% of the dates with averages **above** the averages of all providers: 
 ['Biosynth AG', 'Pharmeks, Ltd.', 'UkrOrgSynthesis'] 

Number of providers consistently **above** all providers:  3 

----------------------------------------------------------------------- 


Providers that have at least 90% of the dates with averages **equal** the averages of all providers: 
 [] 

Number of providers consistently **equal** all providers:  0 

----------------------------------------------------------------------- 


Providers that have at least 90% of the dates with averages **below** the averages of all providers: 
 ['Manchester Organics Limited', 'Maybridge, Ltd.', 'Otava, Ltd.', 'Otava, Ltd. (2 weeks)'] 

Number of providers consistently **below** all providers:  4 

